In [ ]:
# hide

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
# default_exp django.asgi

<IPython.core.display.Javascript object>

## Old ASGI file

```python
import os


from django.core.asgi import get_asgi_application

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "will_it_saturate.django.settings")

application = get_asgi_application()

```

## Modified Version to Serve Files

In [ ]:
# export
"""
ASGI config for will_it_saturate project.
It exposes the ASGI callable as a module-level variable named ``application``.
For more information on this file, see
https://docs.djangoproject.com/en/3.1/howto/deployment/asgi/
"""

import os
import aiofiles

import django

from asgiref.sync import sync_to_async
from django.core.handlers.asgi import ASGIHandler

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "will_it_saturate.django.settings")

django.setup(set_prefix=False)


class MyASGIHandler(ASGIHandler):
    async def send_response(self, response, send):
        """Encode and send a response out over ASGI."""
        # Collect cookies into headers. Have to preserve header case as there
        # are some non-RFC compliant clients that require e.g. Content-Type.
        response_headers = []
        for header, value in response.items():
            if isinstance(header, str):
                header = header.encode("ascii")
            if isinstance(value, str):
                value = value.encode("latin1")
            response_headers.append((bytes(header), bytes(value)))
        for c in response.cookies.values():
            response_headers.append(
                (b"Set-Cookie", c.output(header="").encode("ascii").strip())
            )
        # Initial response message.
        await send(
            {
                "type": "http.response.start",
                "status": response.status_code,
                "headers": response_headers,
            }
        )
        # Streaming responses need to be pinned to their iterator.
        if response.streaming:
            # print("serving streaming response: ", response.async_file_response, response)
            # print("response headers: ", response_headers)
            if response.async_file_response:
                # print("filename: ", response.filename)
                async with aiofiles.open(response.filename, mode="rb") as file:
                    more_body = True
                    while more_body:
                        chunk = await file.read(response.chunk_size)
                        more_body = len(chunk) == response.chunk_size
                        await send(
                            {
                                "type": "http.response.body",
                                "body": chunk,
                                "more_body": more_body,
                            }
                        )
            else:
                # Access `__iter__` and not `streaming_content` directly in case
                # it has been overridden in a subclass.
                for part in response:
                    for chunk, _ in self.chunk_bytes(part):
                        await send(
                            {
                                "type": "http.response.body",
                                "body": chunk,
                                # Ignore "more" as there may be more parts; instead,
                                # use an empty final closing message with False.
                                "more_body": True,
                            }
                        )
                # Final closing message.
                await send({"type": "http.response.body"})
        # Other responses just need chunking.
        else:
            # Yield chunks of response.
            for chunk, last in self.chunk_bytes(response.content):
                await send(
                    {
                        "type": "http.response.body",
                        "body": chunk,
                        "more_body": not last,
                    }
                )
        await sync_to_async(response.close, thread_sensitive=True)()


# application = MyASGIHandler()

<IPython.core.display.Javascript object>

In [ ]:
# export


class MyASGIHandler2(ASGIHandler):
    async def send_response(self, response, send):
        """Encode and send a response out over ASGI."""
        # Collect cookies into headers. Have to preserve header case as there
        # are some non-RFC compliant clients that require e.g. Content-Type.
        response_headers = []
        for header, value in response.items():
            if isinstance(header, str):
                header = header.encode("ascii")
            if isinstance(value, str):
                value = value.encode("latin1")
            response_headers.append((bytes(header), bytes(value)))
        for c in response.cookies.values():
            response_headers.append(
                (b"Set-Cookie", c.output(header="").encode("ascii").strip())
            )
        # Initial response message.
        await send(
            {
                "type": "http.response.start",
                "status": response.status_code,
                "headers": response_headers,
            }
        )
        # Streaming responses need to be pinned to their iterator.
        if response.streaming:
            if response.async_file_response:
                print("filename: ", response.filename)
                async for more_body, chunk in response:
                    await send(
                        {
                            "type": "http.response.body",
                            "body": chunk,
                            "more_body": more_body,
                        }
                    )

            else:
                # Access `__iter__` and not `streaming_content` directly in case
                # it has been overridden in a subclass.
                for part in response:
                    for chunk, _ in self.chunk_bytes(part):
                        await send(
                            {
                                "type": "http.response.body",
                                "body": chunk,
                                # Ignore "more" as there may be more parts; instead,
                                # use an empty final closing message with False.
                                "more_body": True,
                            }
                        )
                # Final closing message.
                await send({"type": "http.response.body"})
        # Other responses just need chunking.
        else:
            # Yield chunks of response.
            for chunk, last in self.chunk_bytes(response.content):
                await send(
                    {
                        "type": "http.response.body",
                        "body": chunk,
                        "more_body": not last,
                    }
                )
        await sync_to_async(response.close, thread_sensitive=True)()

<IPython.core.display.Javascript object>

In [ ]:
# export
application = MyASGIHandler2()

<IPython.core.display.Javascript object>

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_views_for_fastapi_server.ipynb.
Converted 12_views_for_django_server.ipynb.
Converted 13_django_asgi.ipynb.
Converted 14_django_async_file_response.ipynb.
Converted 15_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_remote_control_server.ipynb.
Converted 52_benchmark_from_config.ipynb.
Converted 52_generate_files_for_benchmark.ipynb.
Converted 60_legacy.ipynb.
Converted 61_run_benchmark.ipynb.


<IPython.core.display.Javascript object>